In [2]:
import overpy
import csv
import os
from tqdm import tqdm

import roadmaptools.inout

INFO:root:Loading config for project RoadmaptoolsConfig (can be overwritten later)
INFO:root:map_dir: FILL YOUR MAP DIR HERE
osm_source_url: https://s3.amazonaws.com/metro-extracts.mapzen.com/valencia_spain.osm.bz2
osm_map_filename: FILL YOUR MAP DIR HEREmap.osm
filtered_osm_filename: FILL YOUR MAP DIR HEREmap-filtered.osm
geojson_file: FILL YOUR MAP DIR HEREmap.geojson
cleaned_geojson_file: FILL YOUR MAP DIR HEREmap-cleaned.geojson
sanitized_geojson_file: FILL YOUR MAP DIR HEREmap-sanitized.geojson
simplified_file: FILL YOUR MAP DIR HEREmap-simplified.geojson
simplified_file_with_speed: FILL YOUR MAP DIR HEREmap-simplified-speed.geojson
simplified_file_with_speed_and_curvature: FILL YOUR MAP DIR HEREmap-simplified-speed-curvature.geojson
ap_nodes_file: FILL YOUR MAP DIR HEREnodes.geojson
ap_edges_file: FILL YOUR MAP DIR HEREedges.geojson
utm_center_lon: 50.0877506
utm_center_lat: 14.4209293
shift_utm_coordinate_origin_to_utm_center: False
shapely_error_tolerance: 0.005
osm_filter_para

In [3]:
node_file = r"C:\AIC Experiment Data\VGA Manhattan\maps/nodes-overpass.csv"

In [4]:
api = overpy.Overpass()
result = api.query("""
area[name="New York County"];
way(area)[highway~"(motorway|motorway_link|trunk|trunk_link|primary|primary_link|secondary|secondary_link|tertiary|tertiary_link|unclassified|unclassified_link|residential|residential_link|living_street)"][access!="no"];
node(w);
out body;
""")

In [8]:
result.relations

[]

In [11]:
with open(node_file, 'w') as out:
    for node in result.nodes:
         out.write("{},{},{}\n".format(node.id, node.lat, node.lon))

In [55]:
def coord_string_to_E6(coord: str):
    return int(round(float(coord) * 1E7))

In [56]:
node_dict = {}
with open(node_file, 'r') as input_file:
    reader = csv.reader(input_file)
    print("Loading csv file from: {}".format(os.path.realpath(node_file)))
    for row in tqdm(reader):
        coords = (coord_string_to_E6(row[1]), coord_string_to_E6(row[2]))
        if coords in node_dict:
            raise Exception("Coords {} are already in node dict".format(coords))
        node_dict[coords] = int(row[0])

27656it [00:00, 354499.86it/s]

Loading csv file from: C:\AIC Experiment Data\VGA Manhattan\maps\nodes-overpass.csv


Exception: Coords (408765657, -739240191) are already in node dict

In [44]:
edata = roadmaptools.inout.load_geojson(r"C:\AIC Experiment Data\VGA Manhattan\maps/edges-wrong_node_ids.geojson")
edges = edata["features"]

[12:09:32]: Loading geojson file from: C:\AIC Experiment Data\VGA Manhattan\maps\edges-wrong_node_ids.geojson


In [45]:
edges

[{"geometry": {"coordinates": [[-73.982011, 40.785593], [-73.980678, 40.785031]], "type": "LineString"}, "id": 5668968, "properties": {"from_id": "27398201140785593", "from_osm_id": 42421778, "highway": "residential", "id": "207398201100407855930207398067800407850309", "lanes": 2, "length": 26200, "maxspeed": 25, "oneway": "yes", "speed_unit": "mph", "to_id": "27398067840785031", "to_osm_id": 42421776, "utm_coords": [[-650505834, 1057962851], [-650514147, 1057938005]]}, "type": "Feature"},
 {"geometry": {"coordinates": [[-73.980678, 40.785031], [-73.979583, 40.784569]], "type": "LineString"}, "id": 5668968, "properties": {"from_id": "27398067840785031", "from_osm_id": 42421776, "highway": "residential", "id": "207398067800407850309207397958300407845690", "lanes": 2, "length": 21523, "maxspeed": 25, "oneway": "yes", "speed_unit": "mph", "to_id": "27397958340784569", "to_osm_id": 42421775, "utm_coords": [[-650514147, 1057938005], [-650520983, 1057917596]]}, "type": "Feature"},
 {"geometr

In [46]:
failed_assignments = 0
for edge in tqdm(edges):
    coords_raw = edge["geometry"]["coordinates"]
    coords_from = (coord_string_to_E6(coords_raw[0][1]), coord_string_to_E6(coords_raw[0][0]))
    if coords_from in node_dict:
        from_id = node_dict[coords_from]
        edge["properties"]["from_osm_id"] = from_id
    else:
        edge["properties"]["from_osm_id"] = "unknown"
        failed_assignments += 1
        
    coords_to = (coord_string_to_E6(coords_raw[1][1]), coord_string_to_E6(coords_raw[1][0]))
    if coords_to in node_dict:
        to_id = node_dict[coords_to]
        edge["properties"]["to_osm_id"] = to_id
    else:
        edge["properties"]["to_osm_id"] = "unknown"
        failed_assignments +=1
    

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9696/9696 [00:00<00:00, 220354.53it/s]


In [47]:
failed_assignments

126

In [31]:
len(edges)

9696

In [11]:
# overpass lib test
import overpass
query = """area[name=Letkov];
way(area)[highway~"(motorway|motorway_link|trunk|trunk_link|primary|primary_link|secondary|secondary_link|tertiary|tertiary_link|unclassified|unclassified_link|residential|residential_link|living_street)"][access!="no"];
out qt;
node(w);
out qt;"""
api = overpass.API(debug=True, timeout=60)
out = api.get(query, verbosity='geom', responseformat="json")


INFO:root:area[name=Letkov];
way(area)[highway~"(motorway|motorway_link|trunk|trunk_link|primary|primary_link|secondary|secondary_link|tertiary|tertiary_link|unclassified|unclassified_link|residential|residential_link|living_street)"][access!="no"];
out qt;
node(w);
out qt;
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): overpass-api.de:443


[out:json];area[name=Letkov];
way(area)[highway~"(motorway|motorway_link|trunk|trunk_link|primary|primary_link|secondary|secondary_link|tertiary|tertiary_link|unclassified|unclassified_link|residential|residential_link|living_street)"][access!="no"];
out qt;
node(w);
out qt;out geom;
send: b'POST /api/interpreter HTTP/1.1\r\nHost: overpass-api.de\r\nUser-Agent: python-requests/2.25.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nAccept-Charset: utf-8;q=0.7,*;q=0.7\r\nContent-Length: 383\r\nContent-Type: application/x-www-form-urlencoded\r\n\r\n'
send: b'data=%5Bout%3Ajson%5D%3Barea%5Bname%3DLetkov%5D%3B%0Away%28area%29%5Bhighway~%22%28motorway%7Cmotorway_link%7Ctrunk%7Ctrunk_link%7Cprimary%7Cprimary_link%7Csecondary%7Csecondary_link%7Ctertiary%7Ctertiary_link%7Cunclassified%7Cunclassified_link%7Cresidential%7Cresidential_link%7Cliving_street%29%22%5D%5Baccess%21%3D%22no%22%5D%3B%0Aout+qt%3B%0Anode%28w%29%3B%0Aout+qt%3Bout+geom%3B'


DEBUG:urllib3.connectionpool:https://overpass-api.de:443 "POST /api/interpreter HTTP/1.1" 200 17609


reply: 'HTTP/1.1 200 OK\r\n'
header: Date: Wed, 21 Apr 2021 15:22:07 GMT
header: Server: Apache/2.4.18 (Ubuntu)
header: Vary: Accept-Encoding
header: Content-Encoding: gzip
header: Content-Length: 17609
header: Keep-Alive: timeout=5, max=100
header: Connection: Keep-Alive
header: Content-Type: application/json
application/json


In [12]:
out

{'version': 0.6,
 'generator': 'Overpass API 0.7.56.9 76e5016d',
 'osm3s': {'timestamp_osm_base': '2021-04-21T15:20:27Z',
  'timestamp_areas_base': '2021-04-21T14:02:12Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'way',
   'id': 27515335,
   'nodes': [302064059,
    39693418,
    302063712,
    1490190098,
    26422060,
    302065207,
    26422061,
    302065350,
    1490181030,
    26422062,
    1490181039,
    302065299,
    26422063,
    1490181086,
    26422066,
    1490181091,
    1490181094,
    26422069,
    1490181113,
    494386715],
   'tags': {'highway': 'motorway',
    'int_ref': 'E 50',
    'lanes': '2',
    'maxspeed': '130',
    'note:reg_name': 'reg_name je převzat z Wikipedie příslušný článek.',
    'oneway': 'yes',
    'ref': 'D5',
    'reg_name': 'Via Carolina',
    'toll': 'no',
    'toll:hgv': 'yes',
    'toll:motorcycle': 'no'}},
  {'type': 'way',
   'id': 407

In [27]:
import osm2geojson
geojson = osm2geojson.json2geojson(out)

In [15]:
geojso

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'type': 'way',
    'id': 27515335,
    'tags': {'highway': 'motorway',
     'int_ref': 'E 50',
     'lanes': '2',
     'maxspeed': '130',
     'note:reg_name': 'reg_name je převzat z Wikipedie příslušný článek.',
     'oneway': 'yes',
     'ref': 'D5',
     'reg_name': 'Via Carolina',
     'toll': 'no',
     'toll:hgv': 'yes',
     'toll:motorcycle': 'no'}},
   'geometry': {'type': 'LineString',
    'coordinates': [[13.4461897, 49.7123536],
     [13.4484489, 49.7133973],
     [13.4505277, 49.7144371],
     [13.4516189, 49.715046],
     [13.4526527, 49.7156641],
     [13.4541244, 49.7166164],
     [13.4564023, 49.718206],
     [13.4582075, 49.7194314],
     [13.4591551, 49.7200473],
     [13.460174, 49.7206449],
     [13.4614388, 49.7213389],
     [13.4627168, 49.7219654],
     [13.4643511, 49.7226927],
     [13.4667435, 49.7236505],
     [13.4692102, 49.7245469],
     [13.4708457, 49.725101],
     [13.472

In [17]:
node_dict = {}
for el in out['elements']:
    if el['type'] == 'way':
        node_dict[el['id']] = el['nodes']

In [18]:
node_dict

{27515335: [302064059,
  39693418,
  302063712,
  1490190098,
  26422060,
  302065207,
  26422061,
  302065350,
  1490181030,
  26422062,
  1490181039,
  302065299,
  26422063,
  1490181086,
  26422066,
  1490181091,
  1490181094,
  26422069,
  1490181113,
  494386715],
 40701785: [494386718,
  1490181115,
  33959111,
  1490181098,
  1490181092,
  33959112,
  1490181088,
  33959113,
  302065317,
  1490181047,
  33959114,
  1490181033,
  302065373,
  33959116,
  302065262,
  33959117,
  1490190100,
  302063711,
  3772108298,
  8334964404,
  3772108293,
  39693422,
  3772108287,
  302064018],
 426102304: [2229461820,
  74075015,
  2229461821,
  2229461822,
  74027564,
  2229461823,
  2229461824,
  2229461825,
  2229461826,
  3772108313,
  73968854,
  2229461827,
  74165868,
  2229461828,
  74089201,
  2229461829,
  74041083,
  73983294,
  74047196,
  73989990,
  73930592,
  74120431,
  1490181020,
  74060902,
  991454468,
  423264055],
 40700744: [494349673,
  3775697713,
  494349675,
  

In [28]:
for feature in geojson['features']:
    if feature['geometry']['type'] == 'LineString':
        feature['id'] = feature['properties']['id']
        del feature['properties']['id']
        for key, val in feature['properties']['tags'].items():
            feature['properties'][key] = val
        del feature['properties']['tags']
        feature['properties']['nodes'] = node_dict[feature['id']]

In [29]:
geojso

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'type': 'way',
    'highway': 'motorway',
    'int_ref': 'E 50',
    'lanes': '2',
    'maxspeed': '130',
    'note:reg_name': 'reg_name je převzat z Wikipedie příslušný článek.',
    'oneway': 'yes',
    'ref': 'D5',
    'reg_name': 'Via Carolina',
    'toll': 'no',
    'toll:hgv': 'yes',
    'toll:motorcycle': 'no',
    'nodes': [302064059,
     39693418,
     302063712,
     1490190098,
     26422060,
     302065207,
     26422061,
     302065350,
     1490181030,
     26422062,
     1490181039,
     302065299,
     26422063,
     1490181086,
     26422066,
     1490181091,
     1490181094,
     26422069,
     1490181113,
     494386715]},
   'geometry': {'type': 'LineString',
    'coordinates': [[13.4461897, 49.7123536],
     [13.4484489, 49.7133973],
     [13.4505277, 49.7144371],
     [13.4516189, 49.715046],
     [13.4526527, 49.7156641],
     [13.4541244, 49.7166164],
     [13.4564023, 49.718206],